In [2]:
import cv2
import mediapipe as mp
import numpy as np
import torch
from tensorflow.keras.models import load_model

# Load the trained fall detection model
model = load_model("/home/aekankpatel/Desktop/Fall_Detection/Model.h5")

# Load MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Define labels for predictions
labels = {0: "Non-Fall", 1: "Fall"}

# Function to extract pose features from an image
def extract_pose_features(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks is None:
        return None
    # Extracting pose landmarks
    landmarks = np.array([[lmk.x, lmk.y] for lmk in results.pose_landmarks.landmark]).flatten()
    return landmarks

# Function to perform real-time fall detection
def detect_fall(frame, threshold=0.5, detections=[]):
    results = yolo_model(frame)
    new_detections = results.xyxy[0].cpu().numpy()  # Extract the bounding boxes

    updated_detections = []
    
    for new_detection in new_detections:
        x1, y1, x2, y2, conf, cls = new_detection
        if int(cls) == 0:  # Person class
            # Extract the person's region of interest (ROI)
            person_roi = frame[int(y1):int(y2), int(x1):int(x2)]
            features = extract_pose_features(person_roi)
            if features is not None:
                # Make prediction
                features = np.expand_dims(features, axis=0)
                prediction = model.predict(features)
                predicted_confidence = prediction[0]  # Get the single prediction value
                predicted_label = labels[1] if predicted_confidence > threshold else labels[0]  # Thresholding
                
                # Convert predicted_confidence to scalar value
                predicted_confidence_scalar = predicted_confidence.item()
                
                # Choose color based on predicted label
                color = (0, 255, 0)  # Green for Non-Fall
                if predicted_label == "Fall":
                    color = (0, 0, 255)  # Red color for Fall
                
                # Check if the detection is overlapping with an existing one
                existing = False
                for detection in detections:
                    if (abs(detection['x1'] - x1) < 20 and abs(detection['y1'] - y1) < 20):
                        # Update the existing detection
                        detection['x1'], detection['y1'], detection['x2'], detection['y2'] = x1, y1, x2, y2
                        detection['label'] = predicted_label
                        detection['confidence'] = predicted_confidence_scalar
                        existing = True
                        break
                
                if not existing:
                    # Add new detection
                    updated_detections.append({
                        'x1': x1,
                        'y1': y1,
                        'x2': x2,
                        'y2': y2,
                        'label': predicted_label,
                        'confidence': predicted_confidence_scalar
                    })
    
    # Draw all detections on the frame
    for detection in detections + updated_detections:
        color = (0, 255, 0) if detection['label'] == "Non-Fall" else (0, 0, 255)
        cv2.rectangle(frame, (int(detection['x1']), int(detection['y1'])), (int(detection['x2']), int(detection['y2'])), color, 2)
        cv2.putText(frame, f"{detection['label']} ({detection['confidence']:.2f})", (int(detection['x1']), int(detection['y1']) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame, updated_detections

# Open webcam
cap = cv2.VideoCapture(0)
detections = []

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Perform fall detection on the frame
    frame, detections = detect_fall(frame, detections=detections)

    # Check if the frame dimensions are valid
    if frame.shape[0] > 0 and frame.shape[1] > 0:
        # Display the resulting frame
        cv2.imshow('Fall Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object fi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━